# Classification cross validation avec indices VGGISH

## 1. Ségrégation spatiale : Tests sur différents modèles

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import re

def separate_integers_letters(string):
    integers = ''.join(re.findall(r'\d+', string))
    letters = ''.join(re.findall(r'[a-zA-Z]+', string))
    return integers, letters

In [3]:
dico_encodage = {'E_Bleu': 1, 'E_Rouge': 2, 'L_Bleu': 3, 'L_Rouge': 4}
print(dico_encodage)

{'E_Bleu': 1, 'E_Rouge': 2, 'L_Bleu': 3, 'L_Rouge': 4}


In [4]:
import pandas as pd

def add_labels(df):

  for index, row in df.iterrows():
    station_name_parts = row['station_name'].split('/')
    position_dynamique = station_name_parts[2]
    position_dynamique_parts = position_dynamique.split('_')
    _, position = separate_integers_letters(position_dynamique_parts[1])
    dynamique = position_dynamique_parts[2]

    position_dynamique = position + '_' + dynamique

    df.loc[index, 'label'] = dico_encodage[position_dynamique]

  return df

In [5]:
from sklearn.metrics import accuracy_score

def custom_cross_validation(model, X_train, y_train, X_test, y_test):
    # Entraîner le modèle sur l'ensemble de données d'entraînement
    model.fit(X_train, y_train)

    # Prédire les étiquettes sur l'ensemble de test
    y_pred = model.predict(X_test)

    # Calculer la précision du modèle
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

In [7]:
import pandas as pd
import os
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Chemin vers le répertoire contenant les dossiers de données sur Google Drive
data_directory = "/content/gdrive/MyDrive/ListenToTheWild/vggish_cross_validation_spatial"

# Liste des dossiers dans le répertoire
directories = sorted(os.listdir(data_directory))

rf_mean_acc = 0
mlp_mean_acc = 0
gb_mean_acc = 0
# Boucle sur chaque dossier
for directory in directories:
  # Chemin vers le dossier contenant les fichiers de train/test
  directory_path = os.path.join(data_directory, directory)

  # Charger les données d'entraînement
  train_data = pd.read_csv(os.path.join(directory_path, f"{directory}_train.csv"))
  nb_rows_train = train_data.shape[0]
  # Ajouter les labels
  train_data_with_labels = add_labels(train_data)
  # Supprimer les colonnes "station_name" et "file_name"
  train_data_with_labels = train_data_with_labels.drop(['station_name', 'file_name'], axis=1)
  # Créer X et y
  X_train = train_data_with_labels.drop(columns=['label'])  # attributs
  y_train = train_data_with_labels['label']  # étiquettes

  # Charger les données de test
  test_data = pd.read_csv(os.path.join(directory_path, f"{directory}_test.csv"))
  nb_rows_test = test_data.shape[0]
  # Ajouter les labels
  test_data_with_labels = add_labels(test_data)
  # Supprimer les colonnes "station_name" et "file_name"
  test_data_with_labels = test_data_with_labels.drop(['station_name', 'file_name'], axis=1)
  # Créer X et y
  X_test = test_data_with_labels.drop(columns=['label'])  # attributs
  y_test = test_data_with_labels['label']  # étiquettes

  # Créer les modèles
  rf_model = RandomForestClassifier()
  mlp_model = MLPClassifier()
  gb_model = GradientBoostingClassifier()

  # Effectuer les validations croisées personnalisées
  rf_accuracy = custom_cross_validation(rf_model, X_train, y_train, X_test, y_test)
  mlp_accuracy = custom_cross_validation(mlp_model, X_train, y_train, X_test, y_test)
  gb_accuracy = custom_cross_validation(gb_model, X_train, y_train, X_test, y_test)

  # Calculer la moyenne des précisions
  mean_accuracy = (rf_accuracy + mlp_accuracy + gb_accuracy)/3

  # Ajouter les valeurs pour le calcul des moyennes
  rf_mean_acc += rf_accuracy
  mlp_mean_acc += mlp_accuracy
  gb_mean_acc += gb_accuracy

  # Calculer les pourcentages des ensembles
  per_train_set = nb_rows_train / (nb_rows_train + nb_rows_test)
  per_test_set = nb_rows_test / (nb_rows_train + nb_rows_test)

  # Afficher les précisions sur l'ensemble de test
  print(f"Dossier {directory} (train : {round(per_train_set*100, 2)}% / test : {round(per_test_set*100, 2)}%) :")
  print(f"Précision sur l'ensemble de test avec RF : {round(rf_accuracy, 2)}")
  print(f"Précision sur l'ensemble de test avec MLP : {round(mlp_accuracy, 2)}")
  print(f"Précision sur l'ensemble de test avec GB : {round(gb_accuracy, 2)}")
  print(f"Moyenne des précisions : {round(mean_accuracy, 2)}")
  print()

print("=======================================================================")
print()

# Afficher les moyennes des précisions par modèle
nb_dir = len(directories)
print(f"Moyenne des précisions avec RF : {round(rf_mean_acc/nb_dir, 2)}")
print(f"Moyenne des précisions avec MLP : {round(mlp_mean_acc/nb_dir, 2)}")
print(f"Moyenne des précisions avec GB : {round(gb_mean_acc/nb_dir, 2)}")

Dossier Antras_Isard (train : 95.46% / test : 4.54%) :
Précision sur l'ensemble de test avec RF : 0.33
Précision sur l'ensemble de test avec MLP : 0.36
Précision sur l'ensemble de test avec GB : 0.4
Moyenne des précisions : 0.36

Dossier Appy_Etang_dAppy (train : 97.81% / test : 2.19%) :
Précision sur l'ensemble de test avec RF : 0.31
Précision sur l'ensemble de test avec MLP : 0.42
Précision sur l'ensemble de test avec GB : 0.34
Moyenne des précisions : 0.36

Dossier Arrien_Pic_Midi_Bordes (train : 95.68% / test : 4.32%) :
Précision sur l'ensemble de test avec RF : 0.41
Précision sur l'ensemble de test avec MLP : 0.39
Précision sur l'ensemble de test avec GB : 0.42
Moyenne des précisions : 0.41

Dossier Ascou_Pailheres (train : 98.02% / test : 1.98%) :
Précision sur l'ensemble de test avec RF : 0.24
Précision sur l'ensemble de test avec MLP : 0.24
Précision sur l'ensemble de test avec GB : 0.23
Moyenne des précisions : 0.24

Dossier Aston_Plateau_de_Beille (train : 98.14% / test : 1.8

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Dossier Couflens_Pouill (train : 99.08% / test : 0.92%) :
Précision sur l'ensemble de test avec RF : 0.41
Précision sur l'ensemble de test avec MLP : 0.46
Précision sur l'ensemble de test avec GB : 0.34
Moyenne des précisions : 0.4

Dossier Couflens_Saubé (train : 99.34% / test : 0.66%) :
Précision sur l'ensemble de test avec RF : 0.35
Précision sur l'ensemble de test avec MLP : 0.75
Précision sur l'ensemble de test avec GB : 0.32
Moyenne des précisions : 0.47



/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Dossier Formiguères_Serra_dels_Alarbs (train : 95.7% / test : 4.3%) :
Précision sur l'ensemble de test avec RF : 0.38
Précision sur l'ensemble de test avec MLP : 0.29
Précision sur l'ensemble de test avec GB : 0.4
Moyenne des précisions : 0.35



...

Dossier Les_Angles_Puig_del_Pam (train : 95.33% / test : 4.67%) :
Précision sur l'ensemble de test avec RF : 0.29
Précision sur l'ensemble de test avec MLP : 0.25
Précision sur l'ensemble de test avec GB : 0.3
Moyenne des précisions : 0.28

Dossier Les_Angles_Roc_dAude (train : 95.42% / test : 4.58%) :
Précision sur l'ensemble de test avec RF : 0.34
Précision sur l'ensemble de test avec MLP : 0.33
Précision sur l'ensemble de test avec GB : 0.34
Moyenne des précisions : 0.34



/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Dossier Merens_Comte (train : 95.99% / test : 4.01%) :
Précision sur l'ensemble de test avec RF : 0.17
Précision sur l'ensemble de test avec MLP : 0.27
Précision sur l'ensemble de test avec GB : 0.12
Moyenne des précisions : 0.19

Dossier Miglos_Pla_de_Montcamp (train : 98.14% / test : 1.86%) :
Précision sur l'ensemble de test avec RF : 0.22
Précision sur l'ensemble de test avec MLP : 0.26
Précision sur l'ensemble de test avec GB : 0.22
Moyenne des précisions : 0.23

Dossier Mijanes_Estagnet (train : 96.3% / test : 3.7%) :
Précision sur l'ensemble de test avec RF : 0.26
Précision sur l'ensemble de test avec MLP : 0.24
Précision sur l'ensemble de test avec GB : 0.22
Moyenne des précisions : 0.24

Dossier Mijanes_Trabesses (train : 97.3% / test : 2.7%) :
Précision sur l'ensemble de test avec RF : 0.29
Précision sur l'ensemble de test avec MLP : 0.23
Précision sur l'ensemble de test avec GB : 0.29
Moyenne des précisions : 0.27

Dossier Montferrier_Mont_dOlmes (train : 97.46% / test : 2.54

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Dossier Saint-Lary_Herbe_Soulette (train : 98.02% / test : 1.98%) :
Précision sur l'ensemble de test avec RF : 0.37
Précision sur l'ensemble de test avec MLP : 0.37
Précision sur l'ensemble de test avec GB : 0.34
Moyenne des précisions : 0.36

Dossier Seix_Col_de_Pause (train : 98.23% / test : 1.77%) :
Précision sur l'ensemble de test avec RF : 0.5
Précision sur l'ensemble de test avec MLP : 0.25
Précision sur l'ensemble de test avec GB : 0.53
Moyenne des précisions : 0.43

Dossier Seix_Pic_de_Montbuou (train : 95.73% / test : 4.27%) :
Précision sur l'ensemble de test avec RF : 0.47
Précision sur l'ensemble de test avec MLP : 0.51
Précision sur l'ensemble de test avec GB : 0.48
Moyenne des précisions : 0.49

Dossier Sentein_Eylie (train : 95.89% / test : 4.11%) :
Précision sur l'ensemble de test avec RF : 0.31
Précision sur l'ensemble de test avec MLP : 0.35
Précision sur l'ensemble de test avec GB : 0.32
Moyenne des précisions : 0.33

Dossier Ustou_Col_dEscots (train : 95.93% / test :

## 2. Ségrégation temporelle : Tests sur différents modèles

In [9]:
import pandas as pd
import os
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Chemin vers le répertoire contenant les dossiers de données sur Google Drive
data_directory = "/content/gdrive/MyDrive/ListenToTheWild/vggish_cross_validation_temporal"

# Liste des dossiers dans le répertoire
directories = os.listdir(data_directory)

rf_mean_acc = 0
mlp_mean_acc = 0
gb_mean_acc = 0
# Boucle sur chaque dossier
for directory in directories:
  # Chemin vers le dossier contenant les fichiers de train/test
  directory_path = os.path.join(data_directory, directory)

  # Charger les données d'entraînement
  train_data = pd.read_csv(os.path.join(directory_path, f"{directory}_train.csv"))
  nb_rows_train = train_data.shape[0]
  # Ajouter les labels
  train_data_with_labels = add_labels(train_data)
  # Supprimer les colonnes "station_name" et "file_name"
  train_data_with_labels = train_data_with_labels.drop(['station_name', 'file_name'], axis=1)
  # Créer X et y
  X_train = train_data_with_labels.drop(columns=['label'])  # attributs
  y_train = train_data_with_labels['label']  # étiquettes

  # Charger les données de test
  test_data = pd.read_csv(os.path.join(directory_path, f"{directory}_test.csv"))
  nb_rows_test = test_data.shape[0]
  # Ajouter les labels
  test_data_with_labels = add_labels(test_data)
  # Supprimer les colonnes "station_name" et "file_name"
  test_data_with_labels = test_data_with_labels.drop(['station_name', 'file_name'], axis=1)
  # Créer X et y
  X_test = test_data_with_labels.drop(columns=['label'])  # attributs
  y_test = test_data_with_labels['label']  # étiquettes

  # Créer les modèles
  rf_model = RandomForestClassifier()
  mlp_model = MLPClassifier()
  gb_model = GradientBoostingClassifier()

  # Effectuer les validations croisées personnalisées
  rf_accuracy = custom_cross_validation(rf_model, X_train, y_train, X_test, y_test)
  mlp_accuracy = custom_cross_validation(mlp_model, X_train, y_train, X_test, y_test)
  gb_accuracy = custom_cross_validation(gb_model, X_train, y_train, X_test, y_test)

  # Calculer la moyenne des précisions
  mean_accuracy = (rf_accuracy + mlp_accuracy + gb_accuracy)/3

  # Ajouter les valeurs pour le calcul des moyennes
  rf_mean_acc += rf_accuracy
  mlp_mean_acc += mlp_accuracy
  gb_mean_acc += gb_accuracy

  # Calculer les pourcentages des ensembles
  per_train_set = nb_rows_train / (nb_rows_train + nb_rows_test)
  per_test_set = nb_rows_test / (nb_rows_train + nb_rows_test)

  # Afficher les précisions sur l'ensemble de test
  print(f"Dossier {directory} (train : {round(per_train_set*100, 2)}% / test : {round(per_test_set*100, 2)}%) :")
  print(f"Précision sur l'ensemble de test avec RF : {round(rf_accuracy, 2)}")
  print(f"Précision sur l'ensemble de test avec MLP : {round(mlp_accuracy, 2)}")
  print(f"Précision sur l'ensemble de test avec GB : {round(gb_accuracy, 2)}")
  print(f"Moyenne des précisions : {round(mean_accuracy, 2)}")
  print()

print("=======================================================================")
print()

# Afficher les moyennes des précisions par modèle
nb_dir = len(directories)
print(f"Moyenne des précisions avec RF : {round(rf_mean_acc/nb_dir, 2)}")
print(f"Moyenne des précisions avec MLP : {round(mlp_mean_acc/nb_dir, 2)}")
print(f"Moyenne des précisions avec GB : {round(gb_mean_acc/nb_dir, 2)}")

Dossier 18_19_20_21_22_23 (train : 74.64% / test : 25.36%) :
Précision sur l'ensemble de test avec RF : 0.61
Précision sur l'ensemble de test avec MLP : 0.27
Précision sur l'ensemble de test avec GB : 0.53
Moyenne des précisions : 0.47

Dossier 12_13_14_15_16_17 (train : 75.05% / test : 24.95%) :
Précision sur l'ensemble de test avec RF : 0.58
Précision sur l'ensemble de test avec MLP : 0.46
Précision sur l'ensemble de test avec GB : 0.52
Moyenne des précisions : 0.52

Dossier 0_1_2_3_4_5 (train : 74.93% / test : 25.07%) :
Précision sur l'ensemble de test avec RF : 0.62
Précision sur l'ensemble de test avec MLP : 0.52
Précision sur l'ensemble de test avec GB : 0.53
Moyenne des précisions : 0.56

Dossier 6_7_8_9_10_11 (train : 75.38% / test : 24.62%) :
Précision sur l'ensemble de test avec RF : 0.57
Précision sur l'ensemble de test avec MLP : 0.41
Précision sur l'ensemble de test avec GB : 0.51
Moyenne des précisions : 0.5


Moyenne des précisions avec RF : 0.6
Moyenne des précisions av